# **Return total price after discount**

In [2]:
CREATE OR REPLACE FUNCTION core.get_final_price(p_price numeric,p_discount numeric)
RETURNS NUMERIC AS $$
BEGIN
    RETURN p_price - ((p_price* p_discount)/100);
END;
$$ LANGUAGE plpgsql;

CREATE FUNCTION

Total execution time: 00:00:00.038

# **Return customer age group**

In [3]:
CREATE FUNCTION customers.age_group(p_age int)
RETURNS text as $$
BEGIN
    IF p_age < 18 THEN
        RETURN 'Minor';
    ELSE
        RETURN 'Adult';
    END IF;
END;
$$ LANGUAGE plpgsql;

CREATE FUNCTION

Total execution time: 00:00:00.003

# **Return all orders placed by a customer**

In [6]:
CREATE OR REPLACE FUNCTION sales.get_customer_orders(p_cust_id int)
RETURNS TABLE(order_id int,amount numeric) AS $$
BEGIN
    RETURN query
        SELECT o.order_id,o.total_amount
        FROM sales.orders o
        WHERE
            o.cust_id = p_cust_id;
END;
$$ LANGUAGE plpgsql;

CREATE FUNCTION

Total execution time: 00:00:00.001

- plpgsql
- sql
- plpythonu
- plperl
- plv8
- pllua
- plruby

In [7]:
select sales.get_customer_orders(1)

(2 row(s) affected)

Total execution time: 00:00:00.025

get_customer_orders
"(97657,5113.50)"
"(144464,1045779.00)"


# Return top N products by sales with dynamic limit

In [10]:
CREATE OR REPLACE FUNCTION products.top_products_by_sales(p_limit int)
RETURNS TABLE(prod_id int,total_revenue numeric) AS $$
BEGIN
    RETURN QUERY
    SELECT
        oi.prod_id,SUM(oi.quantity * oi.unit_price) as total_sales
        FROM sales.order_items as oi
        GROUP BY
            oi.prod_id
        ORDER BY
            SUM(oi.quantity * oi.unit_price) DESC
        LIMIT p_limit;
END;
$$ LANGUAGE plpgsql;



CREATE FUNCTION

Total execution time: 00:00:00

In [12]:
SELECT products.top_products_by_sales(5)

(5 row(s) affected)

Total execution time: 00:00:00.088

top_products_by_sales
"(638,58320234.00)"
"(97,57877387.00)"
"(221,57402368.00)"
"(681,57046196.00)"
"(303,57011266.00)"


In [25]:
SELECT * from products.top_products_by_sales(5)

(5 row(s) affected)

Total execution time: 00:00:00.082

prod_id,total_revenue
638,58320234.00
97,57877387.00
221,57402368.00
681,57046196.00
303,57011266.00


# Predict next stock shortage date

In [19]:
CREATE OR REPLACE FUNCTION products.predict_stock_out(p_prod int,p_store int)
RETURNS text AS $$ 
DECLARE
    current_stock int;
    daily_usage NUMERIC;
    days_left int;
BEGIN
    SELECT pi.stock_qty into current_stock
    FROM
        products.inventory pi
    WHERE
        pi.prod_id = p_prod and pi.store_id = p_store;

    SELECT
        AVG(oi.quantity) INTO daily_usage
        FROM sales.order_items oi
        JOIN sales.orders o on oi.order_id = o.order_id
        WHERE oi.prod_id=p_prod and o.store_id = p_store;
    
    IF daily_usage = 0 THEN
        RETURN 'No Sales. Stock not moving';
    END IF;

    days_left := current_stock / daily_usage;

    RETURN 'Current Stock => '|| current_stock ||    ' Daily Usage => ' || daily_usage  || ' Expected stock-out in ' || days_left || ' days';
END;
$$ LANGUAGE plpgsql;

CREATE FUNCTION

Total execution time: 00:00:00.002

In [24]:
SELECT products.predict_stock_out(2,1)

(1 row(s) affected)

Total execution time: 00:00:00.054

predict_stock_out
Current Stock => 24 Daily Usage => 3.8000000000000000 Expected stock-out in 6days


In [38]:
Select * from stores.departments

(6 row(s) affected)

Total execution time: 00:00:00.002

dept_id,dept_name
1,Sales
2,Operations
3,Marketing
4,Finance
5,HR
6,IT


In [28]:
insert into stores.departments VALUES(7,'Manufacturing')

INSERT 0 1

Total execution time: 00:00:00.002

In [31]:
CREATE OR REPLACE PROCEDURE stores.add_departments(id int,name text)
LANGUAGE plpgsql as $$
BEGIN
    insert into stores.departments VALUES(id,name);
END;
$$;

CREATE PROCEDURE

Total execution time: 00:00:00

In [32]:
CALL stores.add_departments(8,'Siraj')

CALL

Total execution time: 00:00:00.001

In [34]:
CREATE OR REPLACE PROCEDURE stores.delete_departments(id int)
LANGUAGE plpgsql as $$
BEGIN
    delete FROM stores.departments
        WHERE dept_id = id;
END;
$$;

CREATE PROCEDURE

Total execution time: 00:00:00.002

In [37]:
CALL stores.delete_departments(7)

CALL

Total execution time: 00:00:00.001

# Monthly revenue rollup generator with commit/rollback